<a href="https://colab.research.google.com/github/RICHAR-SL/Proyectos/blob/main/Pinta%20mi%20emoci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***este es otro ***

In [1]:
!pip install flask pyngrok


In [2]:
!ngrok config add-authtoken 31SvM5OXEuo6RytCNnvo6a5eoC5_7pzrSkHYayLugRygJyR1q



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
from flask import Flask, request, render_template_string
from flask_ngrok import run_with_ngrok
from transformers import BeitImageProcessor, BeitForImageClassification
from PIL import Image
import torch

# Modelo
processor = BeitImageProcessor.from_pretrained("Tanneru/Facial-Emotion-Detection-BEIT-Large")
model = BeitForImageClassification.from_pretrained("Tanneru/Facial-Emotion-Detection-BEIT-Large")

app = Flask(__name__)
run_with_ngrok(app)  # 🔹 Aquí se encarga solo ngrok

HTML_PAGE = """
<!DOCTYPE html>
<html>
<head>
  <title>Detector de Emociones</title>
</head>
<body>
  <h2>Sube una imagen de rostro</h2>
  <form action="/predict" method="post" enctype="multipart/form-data">
    <input type="file" name="file">
    <input type="submit" value="Predecir">
  </form>
  {% if label %}
    <h3>Emoción predicha: {{ label }}</h3>
  {% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_PAGE)

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return "No se subió ninguna imagen"
    file = request.files["file"]
    image = Image.open(file.stream).convert("RGB")

    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class_idx = outputs.logits.argmax(-1).item()
    label = model.config.id2label[predicted_class_idx]

    return render_template_string(HTML_PAGE, label=label)

# 🔹 Con flask_ngrok no pones port
app.run()


ModuleNotFoundError: No module named 'flask_ngrok'

# ***con otro entreenado mas robusto***

In [5]:
!pip install pyngrok


In [6]:
from pyngrok import ngrok
from flask import Flask, request, render_template_string
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch.nn.functional as F

# Modelo HuggingFace
processor = AutoImageProcessor.from_pretrained("trpakov/vit-face-expression")
model = AutoModelForImageClassification.from_pretrained("trpakov/vit-face-expression")

# Flask app
app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def index():
    html = """
    <h1>Detector de Emociones</h1>
    <form method="POST" enctype="multipart/form-data">
        <input type="file" name="image">
        <input type="submit" value="Detectar">
    </form>
    <div>{{ result|safe }}</div>
    """
    result = ""
    if request.method == "POST":
        file = request.files["image"]
        image = Image.open(file.stream).convert("RGB")

        inputs = processor(images=image, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=-1)[0]

        # Top 3 emociones
        top3 = torch.topk(probs, 3)
        result = "<h3>Resultados:</h3><ul>"
        for idx, score in zip(top3.indices, top3.values):
            label = model.config.id2label[idx.item()]
            result += f"<li>{label}: {score.item():.2f}</li>"
        result += "</ul>"

    return render_template_string(html, result=result)

# Crear túnel
public_url = ngrok.connect(5000)
print("🌍 Abre esta URL en tu navegador:", public_url)

app.run(port=5000)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

🌍 Abre esta URL en tu navegador: NgrokTunnel: "https://39f8a270c77f.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Aug/2025 17:59:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Aug/2025 17:59:54] "GET /favicon.ico HTTP/1.1" 404 -
